# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-20-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-20-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-21 04:21:40,33.93911,67.709953,142414,6266,82586.0,53562.0,Afghanistan,365.836294,4.399848
1,NaN,NaN,NaN,Albania,2021-07-21 04:21:40,41.15330,20.168300,132740,2456,130086.0,198.0,Albania,4612.551254,1.850234
2,NaN,NaN,NaN,Algeria,2021-07-21 04:21:40,28.03390,1.659600,155784,3979,107041.0,44764.0,Algeria,355.257228,2.554178
3,NaN,NaN,NaN,Andorra,2021-07-21 04:21:40,42.50630,1.521800,14379,127,13930.0,322.0,Andorra,18609.978645,0.883232
4,NaN,NaN,NaN,Angola,2021-07-21 04:21:40,-11.20270,17.873900,41061,970,34857.0,5234.0,Angola,124.933564,2.362339


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,131586,134653,134653,136643,137853,137853,137853,137853,141489,142414
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132592,132597,132608,132616,132629,132647,132665,132686,132697,132740
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,146064,146942,147883,148797,149906,151103,152210,153309,154486,155784


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,5561,5791,5791,5923,5983,5983,5983,5983,6213,6266
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3836,3851,3867,3882,3895,3910,3922,3938,3956,3979


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,81077,82586,82586,82586,82586,82586,82586,82586,82586,82586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130050,130052,130053,130059,130061,130067,130072,130074,130081,130086
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,101430,102010,102594,103192,103809,104397,104992,105604,106337,107041


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7299,7306,7306,7320,7329,7341,7346,7347,7350,7365
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22267,22337,22337,22464,22556,22642,22734,22802,22876,23032
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2365,2367,2367,2373,2377,2386,2389,2394,2396,2411


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,318,318,318,321,323,323,323,323,323,324
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-21 04:21:40,32.539527,-86.644082,7365,113,NaN,NaN,"Autauga, Alabama, US",13182.623637,1.534284
693,1075.0,Lamar,Alabama,US,2021-07-21 04:21:40,33.779950,-88.096680,1486,38,NaN,NaN,"Lamar, Alabama, US",10764.215864,2.557201
694,1077.0,Lauderdale,Alabama,US,2021-07-21 04:21:40,34.901719,-87.656247,9830,253,NaN,NaN,"Lauderdale, Alabama, US",10600.782927,2.573754


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,34174774,609529,0.0
India,31174322,414482,30353710.0
Brazil,19419437,544180,17371065.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,34174774,609529,0.0,33565245.0,2021-07-21 04:21:40,37.950547,-91.419547
India,31174322,414482,30353710.0,406130.0,2021-07-21 04:21:40,23.088275,81.806127
Brazil,19419437,544180,17371065.0,1504192.0,2021-07-21 04:21:40,-12.669522,-48.480493
France,5952339,111715,409403.0,5431221.0,2021-07-21 04:21:40,6.430808,-34.730285
Russia,5931925,147457,5318062.0,466406.0,2021-07-21 04:21:40,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3875420,64144,0.0
Texas,3049688,52865,0.0
Florida,2450344,38388,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3875420,64144,0.0,3811276.0,2021-07-21 04:21:40,37.843962,-120.728594
Texas,3049688,52865,0.0,2996823.0,2021-07-21 04:21:40,31.660643,-98.653069
Florida,2450344,38388,0.0,2411956.0,2021-07-21 04:21:40,28.940755,-82.700744
New York,2128710,53784,0.0,2074926.0,2021-07-21 04:21:40,42.544151,-75.474183
Illinois,1403508,25816,0.0,1377692.0,2021-07-21 04:21:40,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1271179,24600,0.0
Arizona,Maricopa,570595,10383,0.0
Illinois,Cook,559884,10550,0.0
Florida,Miami-Dade,522734,6472,0.0
Texas,Harris,408628,6660,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1271179,24600,0.0,1246579.0,2021-07-21 04:21:40,34.308284,-118.228241,6037.0
Arizona,Maricopa,570595,10383,0.0,560212.0,2021-07-21 04:21:40,33.348359,-112.491815,4013.0
Illinois,Cook,559884,10550,0.0,549334.0,2021-07-21 04:21:40,41.841448,-87.816588,17031.0
Florida,Miami-Dade,522734,6472,0.0,516262.0,2021-07-21 04:21:40,25.611236,-80.551706,12086.0
Texas,Harris,408628,6660,0.0,401968.0,2021-07-21 04:21:40,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,137853,132686,153309,14273,40805,1268,4756378,227430,32016,653735,...,5455276,378875,119388,4,292779,55845,315641,6981,185649,83619
2021-07-19,141489,132697,154486,14359,40906,1275,4769142,227522,32120,653998,...,5495947,379072,119973,4,293866,60180,315761,6987,186279,85732
2021-07-20,142414,132740,155784,14379,41061,1275,4784219,227716,32266,654314,...,5542635,379376,120631,4,293866,65607,315761,6992,187602,88415


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,5983,2456,3938,127,957,42,101549,4566,914,10729,...,128988,5883,796,1,3377,254,3587,1368,3084,2622
2021-07-19,6213,2456,3956,127,969,42,101955,4569,915,10729,...,129007,5889,801,1,3392,334,3589,1370,3113,2697
2021-07-20,6266,2456,3979,127,970,42,102381,4572,915,10729,...,129109,5896,805,1,3392,334,3589,1371,3138,2747


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,82586,130074,105604,13844,34755,1224,4394695,218331,23944,639573,...,16430,368458,114303,3,275712,10667,310525,4159,171699,53453
2021-07-19,82586,130081,106337,13897,34790,1225,4407815,218427,23959,639700,...,16487,369101,114865,3,276952,11047,310601,4162,173320,55714
2021-07-20,82586,130086,107041,13930,34857,1225,4420995,218529,23965,639831,...,16686,369652,115514,3,276952,11443,310601,4162,174728,58155


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7299,7306,7306,7320,7329,7341,7346,7347,7350,7365
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1478,1479,1479,1481,1482,1482,1483,1485,1485,1486
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9703,9720,9720,9754,9767,9782,9793,9805,9813,9830
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3150,3150,3150,3155,3160,3161,3162,3164,3164,3165
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16448,16454,16454,16483,16513,16541,16561,16574,16587,16630


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,560202,72440,0,908248,362580,3860499,566289,350675,110387,49,...,873685,3041410,422619,24550,4160,684499,460067,164933,679826,63523
2021-07-19,560920,72899,0,909282,365132,3865655,566670,351084,110423,49,...,875628,3044072,424171,24579,4215,685485,461847,165177,680252,63709
2021-07-20,562311,72899,0,910436,367007,3875420,567215,351302,110457,49,...,876543,3049688,424730,24590,4249,686206,462577,165259,680743,63840


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-18        7347   22802    2394  2727   7099    1252   2301   14915   
2021-07-19        7350   22876    2396  2733   7103    1252   2303   14928   
2021-07-20        7365   23032    2411  2747   7108    1252   2315   14948   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-18         3786     1909  ...    2926    716     3283      819   
2021-07-19         3789     1909  ...    2930    721     3284      825   
2021-07-20         3801     1911  ...    2932    722     3288      829   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-18           4882  3835  2335          0      933    664  
2021-07-19           4891  3842  2339          0      935    665  
2021-07-20           4906  3848  2351          0      935    669  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,11443,381,0,18117,5992,64094,6887,8282,1697,0,...,12625,52827,2413,258,32,11477,6042,2919,8209,760
2021-07-19,11443,381,0,18117,6007,64133,6886,8282,1697,0,...,12630,52839,2416,259,32,11483,6043,2919,8212,760
2021-07-20,11460,381,0,18117,6007,64144,6889,8282,1697,0,...,12638,52865,2420,259,33,11487,6056,2919,8223,766


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-18         113     323      60   65    139      42     71     331   
2021-07-19         113     323      60   65    139      42     71     331   
2021-07-20         113     324      60   65    139      42     71     332   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-18          124       47  ...      33     14       31        7   
2021-07-19          124       47  ...      33     14       31        7   
2021-07-20          124       47  ...      33     14       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-18             41    11    13          0       26      6  
2021-07-19             41    11    13          0       26      6  
2021-07-20             43    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.000000,0.000158,0.007220,0.000000,0.002407,0.000000,0.001460,0.000581,0.003668,0.000494,...,0.008849,0.000375,0.004265,0.0,0.003644,0.094957,0.00000,0.000573,0.005290,0.012177
2021-07-19,0.026376,0.000083,0.007677,0.006025,0.002475,0.005521,0.002684,0.000405,0.003248,0.000402,...,0.007455,0.000520,0.004900,0.0,0.003713,0.077626,0.00038,0.000859,0.003394,0.025269
2021-07-20,0.006538,0.000324,0.008402,0.001393,0.003789,0.000000,0.003161,0.000853,0.004545,0.000483,...,0.008495,0.000802,0.005485,0.0,0.000000,0.090179,0.00000,0.000716,0.007102,0.031295


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.000000,0.0,0.004080,0.0,0.003145,0.0,0.001134,0.001535,0.000000,0.000093,...,0.000217,0.000680,0.003783,0.0,0.004462,0.128889,0.000000,0.000000,0.010485,0.013138
2021-07-19,0.038442,0.0,0.004571,0.0,0.012539,0.0,0.003998,0.000657,0.001094,0.000000,...,0.000147,0.001020,0.006281,0.0,0.004442,0.314961,0.000558,0.001462,0.009403,0.028604
2021-07-20,0.008531,0.0,0.005814,0.0,0.001032,0.0,0.004178,0.000657,0.000000,0.000000,...,0.000791,0.001189,0.004994,0.0,0.000000,0.000000,0.000000,0.000730,0.008031,0.018539


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.0,0.000015,0.005829,0.000000,0.000662,0.000000,0.003545,0.000408,0.000209,0.000236,...,0.000061,0.001334,0.004199,0.0,0.004335,0.034426,0.000000,0.000722,0.014050,0.029467
2021-07-19,0.0,0.000054,0.006941,0.003828,0.001007,0.000817,0.002985,0.000440,0.000626,0.000199,...,0.003469,0.001745,0.004917,0.0,0.004497,0.035624,0.000245,0.000721,0.009441,0.042299
2021-07-20,0.0,0.000038,0.006620,0.002375,0.001926,0.000000,0.002990,0.000467,0.000250,0.000205,...,0.012070,0.001493,0.005650,0.0,0.000000,0.035847,0.000000,0.000000,0.008124,0.043813


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,0.001294,0.000000,NaN,0.002353,0.000000,0.000546,0.000813,0.000000,0.000435,0.0,...,0.000848,0.000574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-07-19,0.001282,0.006336,NaN,0.001138,0.007038,0.001336,0.000673,0.001166,0.000326,0.0,...,0.002224,0.000875,0.003672,0.001181,0.013221,0.001440,0.003869,0.001479,0.000627,0.002928
2021-07-20,0.002480,0.000000,NaN,0.001269,0.005135,0.002526,0.000962,0.000621,0.000308,0.0,...,0.001045,0.001845,0.001318,0.000448,0.008066,0.001052,0.001581,0.000496,0.000722,0.002056


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-07-18      0.000136  0.002991  0.002093  0.002205  0.001269     0.0   
2021-07-19      0.000408  0.003245  0.000835  0.002200  0.000563     0.0   
2021-07-20      0.002041  0.006819  0.006260  0.005123  0.000704     0.0   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-18      0.000435  0.000872  0.000529  0.000524  ...  0.000000   
2021-07-19      0.000869  0.000872  0.000792  0.000000  ...  0.001367   
2021-07-20      0.005211  0.001340  0.003167  0.001048  ...  0.000683   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-18      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-07-19      0.006983  0.000305  0.007326   0.001844  0.001825  0.001713   
2021-07-20      0.001387  0.001218  0.004848   0.003067  0.001562  0.005130   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-18            NaN  0.000000  0.000000  
2021-07-19            NaN  0.002144  0.001506  
2021-07-20            NaN  0.000000  0.006015  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,0.000000,0.0,NaN,0.000939,0.000000,0.000016,0.000145,0.0,0.0,NaN,...,0.007341,0.000208,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
2021-07-19,0.000000,0.0,NaN,0.000000,0.002503,0.000608,-0.000145,0.0,0.0,NaN,...,0.000396,0.000227,0.001243,0.003876,0.00000,0.000523,0.000166,0.0,0.000365,0.000000
2021-07-20,0.001486,0.0,NaN,0.000000,0.000000,0.000172,0.000436,0.0,0.0,NaN,...,0.000633,0.000492,0.001656,0.000000,0.03125,0.000348,0.002151,0.0,0.001340,0.007895


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2021-07-18         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-07-19         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-07-20         0.0  0.003096     0.0  0.0    0.0     0.0    0.0  0.003021   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-18          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-19          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-18        0.00000   0.0   0.0        NaN      0.0    0.0  
2021-07-19        0.00000   0.0   0.0        NaN      0.0    0.0  
2021-07-20        0.04878   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.001220,0.000140,0.007287,0.000678,0.002515,0.000124,0.002306,0.000677,0.003824,0.000524,...,0.009224,0.000621,0.004221,5.385290e-28,0.003876,0.096275,0.000149,0.000626,0.006443,0.018664
2021-07-19,0.013798,0.000111,0.007482,0.003352,0.002495,0.002822,0.002495,0.000541,0.003536,0.000463,...,0.008340,0.000571,0.004561,2.692645e-28,0.003794,0.086950,0.000265,0.000743,0.004918,0.021967
2021-07-20,0.010168,0.000218,0.007942,0.002372,0.003142,0.001411,0.002828,0.000697,0.004041,0.000473,...,0.008417,0.000686,0.005023,1.346323e-28,0.001897,0.088565,0.000132,0.000729,0.006010,0.026631


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.001697,5.161916e-11,0.003740,5.471321e-26,0.002825,3.041618e-20,0.002416,0.000969,0.000422,0.000072,...,0.000300,0.001037,0.004384,0.0,0.004012,0.109625,0.000169,0.000367,0.010859,0.022334
2021-07-19,0.020070,2.580958e-11,0.004155,2.735660e-26,0.007682,1.520809e-20,0.003207,0.000813,0.000758,0.000036,...,0.000224,0.001029,0.005333,0.0,0.004227,0.212293,0.000363,0.000915,0.010131,0.025469
2021-07-20,0.014300,1.290479e-11,0.004985,1.367830e-26,0.004357,7.604046e-21,0.003693,0.000735,0.000379,0.000018,...,0.000507,0.001109,0.005163,0.0,0.002113,0.106146,0.000182,0.000822,0.009081,0.022004


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-18,0.000232,0.000026,0.005776,0.000306,0.000992,0.000205,0.003677,0.000455,0.000268,0.000185,...,0.004247,0.001547,0.004187,1.292470e-26,0.004367,0.029972,0.000205,0.000589,0.011326,0.024301
2021-07-19,0.000116,0.000040,0.006359,0.002067,0.001000,0.000511,0.003331,0.000447,0.000447,0.000192,...,0.003858,0.001646,0.004552,6.462349e-27,0.004432,0.032798,0.000225,0.000655,0.010383,0.033300
2021-07-20,0.000058,0.000039,0.006490,0.002221,0.001463,0.000255,0.003161,0.000457,0.000349,0.000198,...,0.007964,0.001569,0.005101,3.231174e-27,0.002216,0.034322,0.000113,0.000328,0.009254,0.038556


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,0.001475,0.000627,NaN,0.001501,0.000863,0.000748,0.000891,0.000100,0.000473,2.129876e-148,...,0.000518,0.000825,0.000387,0.000180,0.002526,0.000153,0.000430,0.000127,0.000080,0.000352
2021-07-19,0.001378,0.003481,NaN,0.001320,0.003951,0.001042,0.000782,0.000633,0.000400,1.064938e-148,...,0.001371,0.000850,0.002030,0.000681,0.007873,0.000797,0.002150,0.000803,0.000353,0.001640
2021-07-20,0.001929,0.001741,NaN,0.001294,0.004543,0.001784,0.000872,0.000627,0.000354,5.324689e-149,...,0.001208,0.001347,0.001674,0.000564,0.007970,0.000924,0.001865,0.000650,0.000538,0.001848


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-18      0.000591  0.003454  0.002032  0.001733  0.001658  0.000250   
2021-07-19      0.000500  0.003350  0.001434  0.001967  0.001111  0.000125   
2021-07-20      0.001270  0.005085  0.003847  0.003545  0.000907  0.000062   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-18      0.001152  0.001016  0.000771  0.001255  ...  0.000354   
2021-07-19      0.001010  0.000944  0.000782  0.000628  ...  0.000861   
2021-07-20      0.003111  0.001142  0.001974  0.000838  ...  0.000772   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-18      0.000963  0.000065  0.000890   0.000364  0.000150  0.000169   
2021-07-19      0.003973  0.000185  0.004108   0.001104  0.000987  0.000941   
2021-07-20      0.002680  0.000701  0.004478   0.002085  0.001275  0.003036   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-18           -1.0  0.000337  0.000002  
2021-07-19           -1.0  0.001240  0.000754  
2021-07-20           -1.0  0.000620  0.003385  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-18,0.000188,0.000046,NaN,0.000669,0.000359,0.000175,0.000599,0.000023,0.000092,NaN,...,0.002785,0.000367,0.000353,4.458752e-08,0.000016,0.000123,0.000309,0.000184,0.000224,0.000188
2021-07-19,0.000094,0.000023,NaN,0.000335,0.001431,0.000392,0.000227,0.000011,0.000046,NaN,...,0.001591,0.000297,0.000798,1.938007e-03,0.000008,0.000323,0.000237,0.000092,0.000295,0.000094
2021-07-20,0.000790,0.000012,NaN,0.000167,0.000716,0.000282,0.000331,0.000006,0.000023,NaN,...,0.001112,0.000395,0.001227,9.690034e-04,0.015629,0.000336,0.001194,0.000046,0.000817,0.003994


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                        \
Admin2               Autauga   Baldwin       Barbour          Bibb   
2021-07-18      2.318511e-15  0.000694  2.525621e-10  3.814697e-06   
2021-07-19      1.159255e-15  0.000347  1.262810e-10  1.907349e-06   
2021-07-20      5.796277e-16  0.001722  6.314051e-11  9.536743e-07   

Province_State                                                          \
Admin2                Blount       Bullock        Butler       Calhoun   
2021-07-18      1.988248e-22  6.611402e-22  1.636315e-18  7.400952e-07   
2021-07-19      9.941242e-23  3.305701e-22  8.181574e-19  3.700476e-07   
2021-07-20      4.970621e-23  1.652851e-22  4.090787e-19  1.510759e-03   

Province_State                      ...       Wyoming                          \
Admin2          Chambers  Cherokee  ...          Park    Platte      Sheridan   
2021-07-18      0.000254  0.000043  ...  3.878276e-06  0.001202  7.838224e-40   
2021-07-19      0.000127  0.000022  ...  1.939138e-06  0.000601  3.919112e-40   
2021-07-20      0.000064  0.000011  ...  9.695689e-07  0.000301  1.959556e-40   

Province_State                                                       \
Admin2              Sublette Sweetwater         Teton         Uinta   
2021-07-18      3.443402e-60   0.000391  2.021099e-13  1.102907e-24   
2021-07-19      1.721701e-60   0.000195  1.010550e-13  5.514537e-25   
2021-07-20      8.608505e-61   0.024488  5.052748e-14  2.757269e-25   

Province_State                                         
Admin2         Unassigned      Washakie        Weston  
2021-07-18       0.752941  2.139444e-52  1.818989e-13  
2021-07-19       0.752941  1.069722e-52  9.094947e-14  
2021-07-20       0.752941  5.348609e-53  4.547474e-14  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
